- **BeautifulSoup** : Parse the HTML elements.
- **pandas** : To create and modify the dataframes.
- **requests** : To get access of a webpage.
- **time** : To get the current time.
- **datetime** : To modify date format.
- **gspread** : To access google sheets through it's API.
- **oauth2client** : To access google sheets through it's API.
- **concurrent.futures** : To execute multiple links at once.


In [ ]:
# Importing all required libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime
import time
import gspread
import concurrent.futures


#### We have created a google user account for accessing the Google Sheets API.
- **creds** : Contains the credentials required to access the sheet
- **client** : Authorises our credentials

In [ ]:
# Installing gspread and setup of client
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("creds2020.json",scope)
client = gspread.authorize(creds)

- **sheet** : The sheet we want to access
- **target_sheet** : The worksheet we want to edit in a particular sheet
- **target_sheet.update_acell('A1' , 'Your_text')** : To check that data get feed in right location

In [ ]:
# Assingment of the worksheet No.1 and dry check
sheet = client.open("TOI2020")
target_sheet = sheet.get_worksheet(0)
target_sheet.update_acell('A41504' , 'DateX')


## A. Structure of webpages
### 1. Constants Defined
- **START_TIME** : Unique code of webpage for everyday's archieve. (Here : Start of 2020's 1st Half)
- **INDEX_START** : Index of the dataframe and google sheets
- **LIMIT** : Unique code of webpage for everyday's archieve. (Here : End of 2020's 1st Half)
- **BATCH** : Index for days processed till now (Helps us to trace last update, if any error occurs)
- **NEW_CELL** : Location of cell in which we will update our data.
- **RETRY_DELAY** : Delay in seconds before retrying
- **MAX_RETRY_COUNT** : Maximum number of retries before giving up


### 2. Archieve of Single day
- All article of same day are stored in a table with class = "_rightColWrap_"
- We can access all these links in table through function - **def get_links(daily_url)**

### 3. Individual Article
- We Can access all these data elements through function - **def extract_data(links)**
- Heading : Can access directly, as whole webpage has only one Heading.
- Date : Date is stored in div with class = "_VEOUU_".
- Text : Text is stored in div with class = "*_s30J clearfix*"

## B. Working of different functions
1. **get_links(daily_url)** :</br> To access all individual links in a single day's archieve.
2. **extract_data(links)** :</br> To access all data elements of a single article.
3. **update_excel(final_dataframe, new_cell)** : </br> To update the dataframe to Google sheet.
4. **Try and Except** : </br> try and except are used to encounter error by its own. The *try* will execute the code but this throws an error then *except* will execute code for error handling (given by the user).
5. **except requests.exceptions.RequestException as e** : </br> This will handle any connection                       error occurs (connection lost at host or at client).
6. **except Exception as e** : </br> This will identify the broken links and skip these links. </br>  &nbsp;  &nbsp;&nbsp;1. Link is not working. </br>  &nbsp; &nbsp; 2. The link redirects to third-party links or other publisher's website. </br> &nbsp; &nbsp; 3. The webpage is temporarily or ermanently removed.
7. **with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor** : </br> This will exceute multiple links at once to boost the speed of our task.</br> *max_workers = 5*, is the number of cores used to perform the task (Genrally the value is taken 4-10).
8. **time.sleep(RETRY_DELAY)** : </br> This gives us a window of some seconds to reconnect to the internet by adding some time delay when the connection is lost.


## B. Working of basic functions
1. **response = requests.get(daily_url)** : Sending an access request to the webpage.
2. **html_content = response.text** : Parsing the response in text format.
3. **soup = BeautifulSoup(html_content, 'html.parser')** : Parsing HTML elements and navigating through them.
4. **link = soup.find('ABC', class_="ABC")** : Finding a div with a particular class.
5. **links = link.find_all('a')** :  Finding links stored in the specific div class element.4
6. **url.get('href')** : Accessing all parsed links as hyperlink to use them.
7. **soup.find_all(['ABC'])** : Finding all ABC elements present in our target set.

In [ ]:
# Main Code

# Constants
START_TIME = 43831 
INDEX_START = 12502
LIMIT = 44742
BATCH = 83
# BATCH_SIZE = 25
NEW_CELL = 'A41504'
RETRY_DELAY = 50  
MAX_RETRY_COUNT = 10 

# Function to retrieve links from daily URL
def get_links(daily_url):
    try:
        response = requests.get(daily_url)
        response.raise_for_status()
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        link = soup.find('tr', class_="rightColWrap") 
        links = link.find_all('a')
        return [url.get('href') for url in links]
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving links from daily URL: {e}")
        return []

# Function to extract data from link
def extract_data(links):
    for i in range(MAX_RETRY_COUNT):
        try:
            response = requests.get(links)
            response.raise_for_status()
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')
            try:
                # Extarcting Heading
                headings = [tag.text.strip() for tag in soup.find_all(['h1'])]
                heading = sorted(headings)[0]

                # Extracting Date
                datex = soup.find('div', class_='VEOUU').text
                i = 18  # i = 18, necessary for required format 
                date = "" # defining an empty string

                while (i < len((datex))):
                    date = date + datex[i]
                    i += 1

                # Converting Date from January 1, 2021 to 01-01-2021
                date_str = date 
                date_obj = datetime.strptime(date_str, " %B %d, %Y") #this will tell variable about the location of different elements of date. %B = January
                formatted_date_str = date_obj.strftime("%d-%m-%Y") #arranging the date elements in required format
                Date = formatted_date_str

                # Extracting Text 
                text = soup.find('div', class_='_s30J clearfix').text
                return Date, heading, text
            except Exception as e:
                print(f"Error extracting data from link: {e}")
                return None, None, None
        except requests.exceptions.RequestException as e:
            print(f"Error extracting data from link: {e}")
            time.sleep(RETRY_DELAY)
    return None, None, None

# Function to update Google Sheets
def update_excel(final_dataframe, new_cell):
    try:
        if new_cell == 'A1':
            target_sheet.update(new_cell, [final_dataframe.columns.values.tolist()] + final_dataframe.fillna(-1).values.tolist())
        else:
            target_sheet.update(new_cell, [final_dataframe.columns.values.tolist()] + final_dataframe.fillna(-1).values.tolist())
        print('DataFrame is updated to Excel File successfully.')
    except Exception as e:
        print(f"Error updating Excel file: {e}")

# Main loop
my_columns = ['Date', 'Heading', 'Text']
final_dataframe = pd.DataFrame(columns=my_columns)

t = START_TIME + BATCH
index = INDEX_START
batch = BATCH
run = 0

while t <= LIMIT:
    daily_url = f'https://timesofindia.indiatimes.com/2021/1/1/archivelist/year-2021,month-1,starttime-{t}.cms'
    batch += 1
    t += 1

    links = get_links(daily_url)
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(extract_data, link) for link in links]

        # Add time delay after each error
        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()
                if result:
                    Date, heading, text = result
                    index += 1
                    final_dataframe.loc[index] = [Date, heading, text]
                    run += 1
                    print(f"Batch - {batch}, Entries - {index}, Run - {run}")

                    if run % 10 == 0:  # Update Excel file every 10 runs
                        update_excel(final_dataframe, NEW_CELL)
            except (requests.ConnectionError, requests.Timeout, requests.HTTPError) as e:
                print(f"Error connecting to link: {e}")
                time.sleep(RETRY_DELAY)

print("COMPLETED HALF-1")

##### We are using two separate worksheet, because the google sheets get slow if overloaded and might show some error.

In [ ]:
# Assingment of the worksheet No.2 and dry check
sheet = client.open("TOI2020")
target_sheet = sheet.get_worksheet(2)
target_sheet.update_acell('A1' , 'DateX')

In [ ]:
# Main Code

# Constants
START_TIME = 44743
INDEX_START = 1
LIMIT = 44196
BATCH = 0
# BATCH_SIZE = 25
NEW_CELL = 'A1'
RETRY_DELAY = 50  # Delay in seconds before retrying
MAX_RETRY_COUNT = 10  # Maximum number of retries before giving up

# Function to retrieve links from daily URL
def get_links(daily_url):
    try:
        response = requests.get(daily_url)
        response.raise_for_status()
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        link = soup.find('tr', class_="rightColWrap")
        links = link.find_all('a')
        return [url.get('href') for url in links]
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving links from daily URL: {e}")
        return []

# Function to extract data from link
def extract_data(links):
    for i in range(MAX_RETRY_COUNT):
        try:
            response = requests.get(links)
            response.raise_for_status()
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')
            try:
                headings = [tag.text.strip() for tag in soup.find_all(['h1'])]
                heading = sorted(headings)[0]

                datex = soup.find('div', class_='VEOUU').text
                i = 18  # i = 18, necessary for required format 
                date = "" # defining an empty string

                while (i < len((datex))):
                    date = date + datex[i]
                    i += 1

                date_str = date # Converting Date from January 1, 2021 to 01-01-2021
                date_obj = datetime.strptime(date_str, " %B %d, %Y") #this will tell variable about the location of different elements of date. %B = January
                formatted_date_str = date_obj.strftime("%d-%m-%Y") #arranging the date elements in required format
                Date = formatted_date_str

                text = soup.find('div', class_='_s30J clearfix').text
                return Date, heading, text
            except Exception as e:
                print(f"Error extracting data from link: {e}")
                return None, None, None
        except requests.exceptions.RequestException as e:
            print(f"Error extracting data from link: {e}")
            time.sleep(RETRY_DELAY)
    return None, None, None

# Function to update Excel file
def update_excel(final_dataframe, new_cell):
    try:
        if new_cell == 'A1':
            target_sheet.update(new_cell, [final_dataframe.columns.values.tolist()] + final_dataframe.fillna(-1).values.tolist())
        else:
            target_sheet.update(new_cell, [final_dataframe.columns.values.tolist()] + final_dataframe.fillna(-1).values.tolist())
        print('DataFrame is updated to Excel File successfully.')
    except Exception as e:
        print(f"Error updating Excel file: {e}")

# Main loop
my_columns = ['Date', 'Heading', 'Text']
final_dataframe = pd.DataFrame(columns=my_columns)

t = START_TIME + BATCH
index = INDEX_START
batch = BATCH
run = 0

while t <= LIMIT:
    daily_url = f'https://timesofindia.indiatimes.com/2021/1/1/archivelist/year-2021,month-1,starttime-{t}.cms'
    batch += 1
    t += 1

    links = get_links(daily_url)
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(extract_data, link) for link in links]

        # Add time delay after each error
        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()
                if result:
                    Date, heading, text = result
                    index += 1
                    final_dataframe.loc[index] = [Date, heading, text]
                    run += 1
                    print(f"Batch - {batch}, Entries - {index}, Run - {run}")

                    if run % 10 == 0:  # Update Excel file every 10 runs
                        update_excel(final_dataframe, NEW_CELL)
            except (requests.ConnectionError, requests.Timeout, requests.HTTPError) as e:
                print(f"Error connecting to link: {e}")
                time.sleep(RETRY_DELAY)

print("COMPLETED HALF-2")